## Informações iniciais

Neste projeto de portifolio, eu utilizo uma base de dados de uma seguradora que reuniu informações sobre os seus clientes juntamente com o interesse de renovar o contrato. E utilizando ciência de dados eu as faço gerar lucro, predizendo quais os clientes mais provaveis de renovar o contrato.

Link da base de dados https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction

Principal métrica utilizada: Log loss.

Modelos do ensemble: XGB e Logistic regression.

Importante! Checar explicações  sobre o threshold e modelos escolhidos.

## Importações

In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

## Leitura de treino e teste

In [2]:
train = pd.read_csv('train.csv')

In [3]:
test = pd.read_csv('test.csv')

## Visualizando as primeiras linhas

In [4]:
train.head(3)

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1


## Pré processamento 

Neste dataset, eu optei por não fazer um pré processamento muito avançado.

In [5]:
X = train.drop(columns=['Response','id'])
#Coluna ID removida por não ter valor perditivo e Response por ser o que procuramos 
Y = train['Response']
#Atribuindo a coluna Response ao Y para ser nossa coluna a ser predita
X = pd.get_dummies(X)
#Comando parecido com o onehotencoder, transformar dados categoricos em colunas

### Consertando desbalanceamento de classes

Por conta do desbalanceamento, e para reduzir o tempo de treino, uma redução de linhas com pouco valor preditivo será feita.

In [6]:
from imblearn.under_sampling import NearMiss
NM = NearMiss()
X,Y = NM.fit_sample(X,Y)

## Definição da baseline

Agora, como temos uma base balanceada, não temos problema em utilizar a porcentagem simples de acertos, dessa forma não caímos no "paradoxo da porcentagem", porém utilizaremos apenas como referência, pois não será nossa principal métrica.

In [7]:
n_splits = 2
aux=0
kf = KFold(n_splits=n_splits,shuffle=True,random_state=0)
for tr, ts in kf.split(X,Y):  
    X_train, Y_train = X.iloc[tr], Y.iloc[tr]
    X_test, Y_test = X.iloc[ts], Y.iloc[ts]
    print('baseline no split ',aux,'=',X_test['Vehicle_Damage_Yes'].value_counts()[1]/X_test.shape[0]*100)
    aux=aux+1

baseline no split  0 = 76.03082851637765
baseline no split  1 = 75.67758509955041


## Métrica escolhida

Para este projeto, eu optei por usar a Logloss como principal por penalizar os erros de predição relativo a probabilidade de pertencer as classes 0 e 1.

Além de ficar acompanhando a acurácia simples para ter uma segunda métrica e a matriz de confusão para observar como seria
a predição simples

### MinMaxScaler

Para deixar os valores entre 0 e 1, o que melhora o desempenho para modelos lineares

In [8]:
from sklearn.preprocessing import MinMaxScaler
MMS = MinMaxScaler()
X[X.columns] = MMS.fit_transform(X[X.columns])

### Renomeando colunas

Pois o XGBoost não aceitam as colunas com determinados caracteres

In [9]:
X = X.rename(columns={'Vehicle_Age_1-2 Year':'Veiculo_1_e_2','Vehicle_Age_< 1 Year':'Veiculo_Menor_1',
                     'Vehicle_Age_> 2 Years':'Veiculo_Maior_2'})

### Como ficou 

In [10]:
X

,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Gender_Female,Gender_Male,Veiculo_1_e_2,Veiculo_Menor_1,Veiculo_Maior_2,Vehicle_Damage_No,Vehicle_Damage_Yes
0,0.476190,1.0,0.557692,0.0,0.000000,0.759259,0.103806,0.0,1.0,1.0,0.0,0.0,0.0,1.0
1,0.412698,1.0,0.538462,0.0,0.000000,0.154321,0.982699,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,0.412698,1.0,0.538462,0.0,0.000000,0.154321,0.034602,0.0,1.0,1.0,0.0,0.0,0.0,1.0
3,0.238095,1.0,0.538462,0.0,0.000000,0.956790,0.826990,1.0,0.0,1.0,0.0,0.0,0.0,1.0
4,0.158730,1.0,0.538462,0.0,0.000000,0.956790,0.934256,1.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93415,0.317460,1.0,0.884615,0.0,0.046282,0.154321,0.619377,0.0,1.0,1.0,0.0,0.0,0.0,1.0
93416,0.333333,1.0,0.538462,0.0,0.064641,0.759259,0.307958,1.0,0.0,1.0,0.0,0.0,0.0,1.0
93417,0.412698,1.0,0.538462,0.0,0.059738,0.759259,0.543253,1.0,0.0,1.0,0.0,0.0,0.0,1.0
93418,0.650794,1.0,0.538462,0.0,0.076730,0.154321,0.446367,1.0,0.0,1.0,0.0,0.0,0.0,1.0


## Testando modelos de machine learning

### LogisticRegression

In [11]:
lr = LogisticRegression(max_iter=1000,penalty='none')
n_splits = 2
kf = KFold(n_splits=n_splits,shuffle=True,random_state=0)

for tr, ts in kf.split(X,Y):
    X_train, Y_train = X.iloc[tr], Y.iloc[tr]
    X_test, Y_test = X.iloc[ts], Y.iloc[ts]
    
    lr.fit(X_train, Y_train)
    prev = lr.predict_proba(X_test)[:,1]
    print("Log loss:",log_loss(Y_test,prev))
    prev = (prev > 0.5).astype(int)
    print('Acurácia',accuracy_score(Y_test,prev))
    print("Matriz de confusão:")
    print(confusion_matrix(Y_test,prev))
    print()

Log loss: 0.2048615473277506
Acurácia 0.9087133376150717
Matriz de confusão:
[[23268     8]
 [ 4256 19178]]

Log loss: 0.20650136695932925
Acurácia 0.9088203810747163
Matriz de confusão:
[[23425     9]
 [ 4250 19026]]



### XGBClassifier

In [12]:
XGB = XGBClassifier()
n_splits = 2
kf = KFold(n_splits=n_splits,shuffle=True,random_state=0)

for tr, ts in kf.split(X,Y):
    X_train, Y_train = X.iloc[tr], Y.iloc[tr]
    X_test, Y_test = X.iloc[ts], Y.iloc[ts]
    
    XGB.fit(X_train, Y_train)
    prev = XGB.predict_proba(X_test)[:,1]
    print("Log loss:",log_loss(Y_test,prev))
    prev = (prev > 0.5).astype(int)
    print('Acurácia',accuracy_score(Y_test,prev))
    print("Matriz de confusão:")
    print(confusion_matrix(Y_test,prev))
    print()

Log loss: 0.200551038584103
Acurácia 0.9089916506101478
Matriz de confusão:
[[22902   374]
 [ 3877 19557]]

Log loss: 0.2019008549011119
Acurácia 0.9080924855491329
Matriz de confusão:
[[22982   452]
 [ 3841 19435]]



### Decision Tree

In [13]:
dt = DecisionTreeClassifier(min_samples_leaf=8)
n_splits = 2
kf = KFold(n_splits=n_splits,shuffle=True,random_state=0)
for tr, ts in kf.split(X,Y):
    X_train, Y_train = X.iloc[tr], Y.iloc[tr]
    X_test, Y_test = X.iloc[ts], Y.iloc[ts]

    dt.fit(X_train, Y_train)
    prev = dt.predict_proba(X_test)[:,1]
    print("Log loss:",log_loss(Y_test,prev))
    prev = (prev > 0.5).astype(int)
    print('Acurácia',accuracy_score(Y_test,prev))
    print("Matriz de confusão:")
    print(confusion_matrix(Y_test,prev))
    print()

Log loss: 0.8635227518820908
Acurácia 0.8891886105758938
Matriz de confusão:
[[21449  1827]
 [ 3349 20085]]

Log loss: 0.8899915084210891
Acurácia 0.8907728537786341
Matriz de confusão:
[[21687  1747]
 [ 3355 19921]]



## Ensemble escolhido

Para o ensemble final, eu optei por uma média ponderada do XGB e Logistic regression, sendo respectivamente 0.9 e 0.1, pois foi a combinação que me deu melhor resultado

In [14]:
lr = LogisticRegression(max_iter=1000,penalty='none')
xgb = XGBClassifier(n_estimators=431,learning_rate=0.09751,max_depth=2,min_child_weight=18)
n_splits = 2
kf = KFold(n_splits=n_splits,shuffle=True,random_state=0)
for tr, ts in kf.split(X,Y):
    X_train, Y_train = X.iloc[tr], Y.iloc[tr]
    X_test, Y_test = X.iloc[ts], Y.iloc[ts]
    
    lr.fit(X_train, Y_train)
    xgb.fit(X_train, Y_train)
    prev = lr.predict_proba(X_test)[:,1]
    prev2 = xgb.predict_proba(X_test)[:,1]
    
    prev3 = prev*0.1+prev2*0.9
        
    print("Log loss:",log_loss(Y_test,prev3))
    prev3 = (prev3 > 0.4).astype(int)
    #Threshold
    print('Acurácia',accuracy_score(Y_test,prev3))
    print("Matriz de confusão:")
    print(confusion_matrix(Y_test,prev3))
    print()

Log loss: 0.19604574945096742
Acurácia 0.9085634767715692
Matriz de confusão:
[[22792   484]
 [ 3787 19647]]

Log loss: 0.19696650902770135
Acurácia 0.9095054592164419
Matriz de confusão:
[[22961   473]
 [ 3754 19522]]



## Observações importantes:

### A logistic regression foi muito boa para detectar falsos positivos
### A decision tree foi conseguiu o menor número de falsos negativos
### O XGB conseguiu equilibrar as previsões, mantendo um meio termo entre falsos positivos e negativos
Combinando o XGB e a logistic regression e utilizando um threshold de 0.4 eu consegui equilibrar melhor, porém tudo depende da ocasião, em casos que os falsos positivos não são bem vindos o melhor seria a logistic regression ou aumentar o threshold.

## Predições na base de teste

Re-treino dos algoritmos

In [15]:
lr_final = LogisticRegression(max_iter=1000,penalty='none')
xgb_final = XGBClassifier(n_estimators=431,learning_rate=0.09751,max_depth=2,min_child_weight=18)

lr_final.fit(X,Y)
xgb_final.fit(X,Y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.09751, max_delta_step=0, max_depth=2,
              min_child_weight=18, missing=nan, monotone_constraints='()',
              n_estimators=431, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

## Pré-processamento na base de teste

In [16]:
df_resultados = pd.DataFrame()

In [17]:
#Guardando o id de cada cliente a ser previsto
df_resultados['id'] = test['id']

#Dropando a coluna teste
test = test.drop(columns=['id'])

#Transformação similar a onehotencoder
test = pd.get_dummies(test)

#Normalizando os dados
test[test.columns] = MMS.transform(test[test.columns])

#Renomeando as colunas
test = test.rename(columns={'Vehicle_Age_1-2 Year':'Veiculo_1_e_2','Vehicle_Age_< 1 Year':'Veiculo_Menor_1',
                     'Vehicle_Age_> 2 Years':'Veiculo_Maior_2'})

## Predição

In [18]:
previsoes_lr_final = lr_final.predict_proba(test)[:,1]
previsoes_xgb_final =xgb_final.predict_proba(test)[:,1]

In [19]:
previsoes_ensemble_final = previsoes_lr_final*0.1+previsoes_xgb_final*0.9
#Média ponderada entre os modelos

previsoes_ensemble_final

array([0.98476972, 0.99991974, 0.99952136, ..., 0.97009897, 0.96144305,
       0.96586568])

In [20]:
df_resultados['Porcentagem'] = previsoes_ensemble_final

In [21]:
prev = (previsoes_ensemble_final > 0.4).astype(int)
df_resultados['Binario'] = prev

In [22]:
df_resultados.shape

(127037, 3)

## Ordenando pela porcentagem

In [23]:
df_resultados = df_resultados.sort_values(by='Porcentagem', ascending = False)
df_resultados

,id,Porcentagem,Binario
65648,446758,0.999989,1
63688,444798,0.999989,1
99125,480235,0.999988,1
31480,412590,0.999987,1
71921,453031,0.999987,1
...,...,...,...
62773,443883,0.000409,0
109870,490980,0.000408,0
27941,409051,0.000397,0
84795,465905,0.000391,0


## Salvando os resultados

In [24]:
df_resultados.to_csv('final.csv',index=False)

## Considerações finais 

Conseguimos utilizar os dados de forma a gerar valor para a empresa, no caso ultrapassamos a baseline fazendo um ensamble. Para projetos futuros há a possibilidade de aplicar deep learning, como também utilizar o stacking de forma a melhorar os resultados.

## Extra: Bayesian optimization para tunar os parametros do XGB

In [ ]:
def treinar_modelo(params):
    n_estimators=params[0]
    learning_rate=params[1]
    max_depth=params[2]
    min_child_weight=params[3]
    
    modelo = XGBClassifier(n_estimators=n_estimators,
                       learning_rate=learning_rate,
                       max_depth=max_depth,
                       min_child_weight=min_child_weight
                       )
    modelo.fit(X_train,Y_train.values.ravel())
    previsoes = modelo.predict_proba(X_test)[:,1]
    log = log_loss(Y_test, previsoes)
    print(params," Log:", log,'\n',)
    return log

In [ ]:
parametros = [(100,1000),
           (0.0001,0.1),
           (1,10),
           (1,20)
        ]
from skopt import gp_minimize
resultados_gp = gp_minimize(treinar_modelo, parametros, random_state=0, verbose=0, n_calls=30)
print('Final', " Parametros: ",resultados_gp.x, " Resultado: ",resultados_gp.fun)